In [2]:
import pandas as pd
import numpy as np
import datetime

# --- 1. CONFIGURAÇÃO DA SIMULAÇÃO ---

N_CLIENTES = 10000     # Número de clientes para simular
N_MESES = 24           # Histórico de 24 meses
MES_INICIO_ESTRESSE = 18 # Mês em que o "evento de estresse" começa
PCT_ESTRESSE = 0.20    # % de clientes bons que sofrerão o evento de estresse

# Probabilidades demográficas baseadas nos relatórios (Serasa & ANBIMA)
# [18-25, 26-40, 41-65, >65]
PROB_IDADE = [0.113, 0.338, 0.354, 0.195]
# [Classe A/B, Classe C, Classe D/E]
PROB_CLASSE = [0.24, 0.47, 0.29]
# [Sudeste, Nordeste, Sul, Norte, C.Oeste]
PROB_REGIAO = [0.43, 0.26, 0.15, 0.08, 0.08]

print(f"Iniciando simulação para {N_CLIENTES} clientes em {N_MESES} meses...")

# --- 2. FUNÇÃO PARA CRIAR OS PERFIS ESTÁTICOS ---
def criar_clientes_estaticos(n_clientes):
    """
    Cria a base inicial de clientes com seus perfis demográficos e financeiros
    que não mudam (ou mudam pouco) ao longo do tempo.
    """
    df_clientes = pd.DataFrame(
        index=np.arange(1, n_clientes + 1),
        columns=[
            'id_cliente', 'idade_faixa', 'classe_social', 'regiao', 
            'perfil_investidor', 'limite_cartao', 'scr_divida_inicial', 
            'grupo_simulacao'
        ]
    )
    
    df_clientes['id_cliente'] = np.arange(1, n_clientes + 1)
    
    # --- Demografia (Baseado nos relatórios) ---
    df_clientes['idade_faixa'] = np.random.choice(
        ['18-25', '26-40', '41-65', '65+'], n_clientes, p=PROB_IDADE
    )
    df_clientes['classe_social'] = np.random.choice(
        ['A/B', 'C', 'D/E'], n_clientes, p=PROB_CLASSE
    )
    df_clientes['regiao'] = np.random.choice(
        ['Sudeste', 'Nordeste', 'Sul', 'Norte', 'Centro-Oeste'], n_clientes, p=PROB_REGIAO
    )
    
    # --- Perfil Financeiro (Definindo seus clientes "estáveis") ---
    # Vamos usar o Perfil ANBIMA para definir quem é "estável"
    probs_perfil = {'A/B': [0.36, 0.22, 0.13, 0.29], # [Diversifica, Caderneta, Economiza, Sem Reserva]
                    'C':   [0.13, 0.22, 0.15, 0.50],
                    'D/E': [0.05, 0.15, 0.09, 0.71]}
    
    df_clientes['perfil_investidor'] = df_clientes['classe_social'].apply(
        lambda x: np.random.choice(['Diversifica', 'Caderneta', 'Economiza', 'Sem Reserva'], p=probs_perfil[x])
    )
    
    # --- Definir Limites e Dívidas Iniciais (Baseado na Classe Social) ---
    limites = {'A/B': (10000, 30000), 'C': (3000, 10000), 'D/E': (500, 3000)}
    dividas = {'A/B': (1000, 5000), 'C': (500, 2000), 'D/E': (0, 500)}
    
    df_clientes['limite_cartao'] = df_clientes['classe_social'].apply(lambda x: np.random.uniform(*limites[x]))
    df_clientes['scr_divida_inicial'] = df_clientes['classe_social'].apply(lambda x: np.random.uniform(*dividas[x]))

    # --- GRUPO DE SIMULAÇÃO (O PULO DO GATO) ---
    # 1. estavel_target: Seus clientes "bons" que VÃO sofrer estresse (Seu alvo)
    # 2. estavel_controle: Clientes "bons" que NÃO vão sofrer estresse (Controle)
    # 3. instavel: Clientes "ruins" (não são seu alvo, mas bons para o modelo aprender)
    
    def definir_grupo(row):
        # Clientes "instáveis" (Perfil Sem Reserva ou Economiza)
        if row['perfil_investidor'] in ['Sem Reserva', 'Economiza']:
            return 'instavel'
        # Clientes "estáveis" (Perfil Diversifica ou Caderneta)
        else:
            # Vamos sortear uma parte deles para o grupo de estresse
            if np.random.rand() < PCT_ESTRESSE:
                return 'estavel_target'
            else:
                return 'estavel_controle'
                
    df_clientes['grupo_simulacao'] = df_clientes.apply(definir_grupo, axis=1)
    
    print("Perfis estáticos criados:")
    print(df_clientes['grupo_simulacao'].value_counts(normalize=True))
    
    return df_clientes.set_index('id_cliente')


# --- 3. CARREGAR DADOS MACRO (Seus arquivos do BC) ---
def carregar_dados_macro():
    """
    Carrega os dados de juros e inadimplência dos arquivos CSV.
    Isso dará contexto macroeconômico para cada mês da simulação.
    """
    try:
        # Juros rotativo (20679), Juros pessoal (20665), Inadimplência PF (21084)
        df_rotativo = pd.read_csv('bcdata.sgs.20679.csv', sep=';', decimal=',', parse_dates=['data'], dayfirst=True)
        df_pessoal = pd.read_csv('bcdata.sgs.20665.csv', sep=';', decimal=',', parse_dates=['data'], dayfirst=True)
        df_inadimp = pd.read_csv('bcdata.sgs.21084.csv', sep=';', decimal=',', parse_dates=['data'], dayfirst=True)
        
        # Limpando e formatando
        df_rotativo['taxa_juros_rotativo'] = df_rotativo['valor'] / 100 # Assumindo que 12798 é 127,98%
        df_pessoal['taxa_juros_pessoal'] = df_pessoal['valor'] / 100 # Assumindo que 23978 é 239,78%
        df_inadimp['taxa_inadimplencia_pf'] = df_inadimp['valor']
        
        # Criando uma base macro mensal
        df_macro = df_rotativo[['data', 'taxa_juros_rotativo']].merge(
            df_pessoal[['data', 'taxa_juros_pessoal']], on='data'
        ).merge(
            df_inadimp[['data', 'taxa_inadimplencia_pf']], on='data'
        )
        
        # Criando um 'mes_id' para fazer o merge com a simulação
        df_macro = df_macro.sort_values('data').reset_index(drop=True)
        df_macro['mes_id'] = (df_macro['data'].dt.year - df_macro['data'].dt.year.min()) * 12 + df_macro['data'].dt.month
        
        # Pegando apenas os N_MESES mais recentes para a simulação
        df_macro = df_macro.tail(N_MESES).reset_index(drop=True)
        df_macro['mes_simulacao'] = np.arange(1, N_MESES + 1)
        
        print(f"\nDados macro carregados. Último mês: {df_macro['data'].max().date()}")
        return df_macro[['mes_simulacao', 'taxa_juros_rotativo', 'taxa_juros_pessoal', 'taxa_inadimplencia_pf']]

    except FileNotFoundError:
        print("\nArquivos CSV do Banco Central não encontrados. Rodando sem dados macro.")
        return None

# --- 4. FUNÇÃO PARA GERAR O HISTÓRICO COMPORTAMENTAL ---
def gerar_historico_longitudinal(df_clientes, df_macro):
    """
    Cria o histórico de transações mês a mês para cada cliente,
    simulando o comportamento de cada 'grupo_simulacao'.
    """
    historico_total = []
    
    # Datas da simulação
    datas_mes = pd.date_range(end=datetime.date.today(), periods=N_MESES, freq='MS')
    
    for id_cliente, perfil in df_clientes.iterrows():
        
        # Parâmetros do cliente
        grupo = perfil['grupo_simulacao']
        limite = perfil['limite_cartao']
        divida_scr_atual = perfil['scr_divida_inicial']
        
        for mes_num in range(1, N_MESES + 1):
            
            # --- Parâmetros base de simulação ---
            gasto_cartao = 0
            pagamento_fatura = 0
            flag_rotativo = False
            
            # --- LÓGICA DE COMPORTAMENTO ---
            
            if grupo == 'instavel':
                # Cliente "ruim": gasta muito e paga parcial aleatoriamente
                gasto_cartao = limite * np.random.uniform(0.5, 1.0)
                if np.random.rand() < 0.3: # 30% de chance de pagar parcial
                    pagamento_fatura = gasto_cartao * np.random.uniform(0.15, 0.5)
                    flag_rotativo = True
                else:
                    pagamento_fatura = gasto_cartao
                # Dívida SCR cresce erraticamente
                divida_scr_atual *= np.random.uniform(0.95, 1.1)

            elif grupo == 'estavel_controle':
                # Cliente "bom" (controle): gasta pouco e paga integral
                gasto_cartao = limite * np.random.uniform(0.2, 0.5)
                pagamento_fatura = gasto_cartao
                flag_rotativo = False
                # Dívida SCR controlada
                divida_scr_atual *= np.random.uniform(0.98, 1.02)
                
            elif grupo == 'estavel_target':
                # Cliente "bom" (alvo): se comporta bem ATÉ o estresse
                if mes_num < MES_INICIO_ESTRESSE:
                    # Comportamento normal
                    gasto_cartao = limite * np.random.uniform(0.2, 0.5)
                    pagamento_fatura = gasto_cartao
                    flag_rotativo = False
                    divida_scr_atual *= np.random.uniform(0.98, 1.02)
                else:
                    # **EVENTO DE ESTRESSE (SUAS FEATURES!)**
                    # 1. Aumento do uso do cartão
                    gasto_cartao = limite * np.random.uniform(0.7, 1.1) # Gasto sobe!
                    # 2. Aumento do endividamento no SCR
                    divida_scr_atual *= np.random.uniform(1.1, 1.4) # Dívida externa sobe!
                    
                    # CONSEQUÊNCIA: Cliente não consegue pagar
                    pagamento_fatura = gasto_cartao * np.random.uniform(0.15, 0.3) # Paga só o mínimo
                    flag_rotativo = True
            
            historico_total.append({
                'id_cliente': id_cliente,
                'mes_referencia': datas_mes[mes_num-1],
                'mes_simulacao': mes_num,
                'gasto_total_cartao': gasto_cartao,
                'pagamento_fatura': pagamento_fatura,
                'utilizacao_limite_cartao': gasto_cartao / limite,
                'flag_rotativo': flag_rotativo,
                'scr_saldo_devedor_total': divida_scr_atual
            })
            
    df_hist = pd.DataFrame(historico_total)
    
    # Juntar dados macroeconômicos ao histórico
    if df_macro is not None:
        df_hist = df_hist.merge(df_macro, on='mes_simulacao', how='left')
        
    return df_hist.merge(df_clientes, on='id_cliente', how='left')

# --- 5. FUNÇÃO PARA CRIAR AS FEATURES DE JANELA MÓVEL (SUAS INSIGHTS) ---
def criar_features_janeladas(df):
    """
    Cria as features de tendência (últimos 3 meses) que o modelo usará.
    Garante o alinhamento correto dos índices para evitar o ValueError.
    """
    print("\nCriando features de janela móvel (tendência)...")
    
    # 1. Ordenar o DataFrame (mantendo a ordenação por cliente e mês)
    df = df.sort_values(['id_cliente', 'mes_referencia'])
    
    # 2. Criar o índice composto (id_cliente + índice numérico)
    # Isso garante que o índice do df corresponda ao índice gerado pelo .groupby()
    df_indexed = df.set_index(['id_cliente', df.index]) # Novo DF temporário com MultiIndex
    
    # 3. Configurar o agrupamento no DataFrame original (para preservar as colunas)
    g = df.groupby('id_cliente')
    
    # --- CÁLCULO DAS FEATURES ---
    
    # Os resultados do .rolling() (gasto_media_3m, scr_media_3m, etc.) 
    # terão o MultiIndex. Usamos o .reset_index(level=0, drop=True) 
    # para retornar apenas a parte do índice que corresponde ao índice original do DF.
    
    # 1. "aumento no uso do cartao de crédito"
    gasto_media_3m = g['gasto_total_cartao'].rolling(window=3, min_periods=1).mean().shift(1).reset_index(level=0, drop=True)
    gasto_ult_mes = g['gasto_total_cartao'].shift(1).reset_index(level=0, drop=True)
    
    df['gasto_crescim_3m'] = (gasto_ult_mes / gasto_media_3m) - 1
    
    # 2. "aumento do endividamento no scr"
    scr_media_3m = g['scr_saldo_devedor_total'].rolling(window=3, min_periods=1).mean().shift(1).reset_index(level=0, drop=True)
    scr_ult_mes = g['scr_saldo_devedor_total'].shift(1).reset_index(level=0, drop=True)
    
    df['scr_crescim_divida_3m'] = (scr_ult_mes / scr_media_3m) - 1
    
    # --- Outras features de suporte importantes ---
    
    df['utilizacao_limite_media_3m'] = g['utilizacao_limite_cartao'].rolling(window=3, min_periods=1).mean().shift(1).reset_index(level=0, drop=True)
    df['contagem_rotativo_3m'] = g['flag_rotativo'].rolling(window=3, min_periods=1).sum().shift(1).reset_index(level=0, drop=True)

    # Preenchendo NaNs e corrigindo infinitos
    df = df.fillna(0)
    df = df.replace([np.inf, -np.inf], 0)
    
    return df
    # ----------------------------------------------------------------------
    # --- PASSO 6: SALVAR A BASE FINAL (RECOMENDADO: PARQUET) ---
    # ----------------------------------------------------------------------
    
    print(f"\nSalvando base de ML final em formato Parquet...")
    try:
        # ... (seu código de salvamento em Parquet/CSV) ...
        df_base_ml.to_parquet('base_simulada_clientes.parquet', index=False)
        print("Sucesso! Base salva em 'base_simulada_clientes.parquet'")
        print("Este formato preserva 100% dos tipos de dados (datas, números, etc.).")

    except ImportError:
        # Plano B: Salva em CSV se a biblioteca Parquet não estiver instalada
        print("Biblioteca 'pyarrow' ou 'fastparquet' não encontrada. Salvando em CSV como alternativa.")
        print("AVISO: CSV não preserva tipos de dados. Considere instalar o pyarrow: pip install pyarrow")
        df_base_ml.to_csv('base_simulada_clientes.csv', index=False, sep=';', decimal=',')
        print("Base salva em 'base_simulada_clientes.csv'")

    # --- FINAL: A BASE DE ML ESTÁ PRONTA (DENTRO DA FUNÇÃO) ---
    print("\n--- Base de ML pronta para treinamento! ---")
    
    # Colunas de features que o modelo usará:
    features_modelo = [
        'gasto_crescim_3m', 'scr_crescim_divida_3m', 'utilizacao_limite_media_3m',
        'contagem_rotativo_3m', 'limite_cartao', 'scr_divida_inicial',
        'taxa_juros_rotativo', 'taxa_inadimplencia_pf', 'idade_faixa',
        'classe_social', 'regiao', 'perfil_investidor'
    ]
    
    target = 'entrou_rotativo_proximo_mes'
    
    # Exibir um exemplo de como ficou a base
    # Filtramos para ver um cliente "estavel_target" que sofreu o estresse
    id_exemplo = df_clientes[df_clientes['grupo_simulacao'] == 'estavel_target'].index[0]
    
    print("\nExemplo de um cliente 'estável_target' (ID: {}):".format(id_exemplo))
    print(df_base_ml[df_base_ml['id_cliente'] == id_exemplo][
        ['mes_referencia', 'gasto_total_cartao', 'pagamento_fatura', 'gasto_crescim_3m', 'scr_crescim_divida_3m', 'entrou_rotativo_proximo_mes']
    ].tail(10)) # Mostra os últimos 10 meses
    
    print("\nDistribuição da variável alvo (Target):")
    print(df_base_ml[target].value_counts(normalize=True))
    
    # A ÚLTIMA LINHA DA FUNÇÃO DEVE SER O RETURN
    return df_base_ml

# --- Executar a simulação ---
# --- Executar a simulação ---
if __name__ == "__main__":
    df_base_ml = main()
    
    # --- FINAL: A BASE DE ML ESTÁ PRONTA ---  <-- Começa o erro aqui
    print("\n--- Base de ML pronta para treinamento! ---")
    
    # Colunas de features que o modelo usará:
    # ... (todo o bloco de análise) ...
    
    # O df_clientes não existe aqui, ele existe SÓ dentro de main()
    id_exemplo = df_clientes[df_clientes['grupo_simulacao'] == 'estavel_target'].index[0] 
    
    # ...
    
    return df_base_ml # <-- ESTE RETURN AQUI CAUSOU O ERRO

Iniciando simulação para 10000 clientes em 24 meses...


SyntaxError: 'return' outside function (3871837840.py, line 323)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# Definir colunas para o modelo
features_categoricas = ['idade_faixa', 'classe_social', 'regiao', 'perfil_investidor']
features_numericas = [
    'gasto_crescim_3m', 'scr_crescim_divida_3m', 'utilizacao_limite_media_3m',
    'contagem_rotativo_3m', 'limite_cartao', 'scr_divida_inicial',
    'taxa_juros_rotativo', 'taxa_inadimplencia_pf'
]
target = 'entrou_rotativo_proximo_mes'

# Criar o pipeline de pré-processamento
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', features_numericas),
        ('cat', OneHotEncoder(handle_unknown='ignore'), features_categoricas)
    ])

# Separar Features (X) e Alvo (y)
X = df_base_ml[features_numericas + features_categoricas]
y = df_base_ml[target]

# Dividir os dados
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Aplicar o pré-processamento
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

# Treinar o Modelo
# Usamos class_weight='balanced' pois a base será desbalanceada (poucos entrarão em rotativo)
model = RandomForestClassifier(random_state=42, class_weight='balanced', n_jobs=-1)
model.fit(X_train_processed, y_train)

# Avaliar
y_pred = model.predict(X_test_processed)
print("\n--- Relatório de Classificação do Modelo ---")
print(classification_report(y_test, y_pred))